# Multi Year score
Given a target year, it computes a score for each image year of adquisition

This score is a constant value for the whole scene (not per-pixel)

In [1]:
import ee
import geetools

In [2]:
geetools.Initialize('credentials_ciefap')

In [3]:
from geetools import tools, collection

In [4]:
import ipygee as ui

In [5]:
from geebap import scores
from geebap.season import Season

In [6]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

In [7]:
col = collection.Landsat8SR()

In [8]:
colEE = col.collection.filterBounds(aoi).map(lambda img: img.addBands(col.ndvi(img)))

In [9]:
start = '2015-11-15'
end = '2019-02-15'

In [10]:
colEE = colEE.filterDate(start, end)

In [11]:
year = 2018

In [12]:
years = list(range(2016, 2020))

## Add YEAR_BAP property

In [13]:
season = Season('11-15', '02-15')

In [14]:
ui.eprint(colEE.size())

In [15]:
date_range_first = season.add_year(years[0])
first_col = colEE.filterDate(date_range_first.start(), date_range_first.end())
first_col = first_col.map(lambda img: img.set('YEAR_BAP', years[0]))
collections = ee.List([first_col.toList(first_col.size())])

rest = ee.List(years[1:])

def wrap(year, thelist):
    thelist = ee.List(thelist)
    year = ee.Number(year)
    date_range = season.add_year(year)
    filtered = colEE.filterDate(date_range.start(), date_range.end())
    # add YEAR_BAP
    filt = filtered.map(lambda img: img.set('YEAR_BAP', year))
    filt_list = filt.toList(filt.size())
    return thelist.add(filt_list)

final_col = ee.List(rest.iterate(wrap, collections))

In [16]:
final_col = ee.ImageCollection.fromImages(final_col.flatten())

## Score Linear

In [17]:
score = scores.MultiYear(season=season, main_year=year)

In [18]:
newcol = score.apply(final_col, target_year=year, year_property='YEAR_BAP')

In [19]:
chart = ui.chart.Image.series(newcol, p, scale=30, bands=['year_score'], label_bands=['linear'])

## Score Gauss

In [20]:
newcol_n = score.apply(final_col, target_year=year, function='gauss')

In [21]:
chart_n = ui.chart.Image.series(newcol_n, p, scale=30, bands=['year_score'], label_bands=['gauss stretch 1'])

## PLOT

In [22]:
both = chart.cat(chart_n)
both.renderWidget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [46]:
# both.dataframe

## Using collection

In [21]:
newcol_map = score.map(final_col)

In [22]:
chart = ui.chart.Image.series(newcol_map, p, scale=30, bands=['score-multi'], label_bands=['linear'])

In [23]:
chart.renderWidget(width='800px')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…